In [1]:
# install dependencies
!pip install pytest pytest-sugar

In [2]:
%%file binarysearchtree.py
import plotly.graph_objs as go

class Node:
    """
    A class representing a node in a binary search tree.

    Attributes:
    - value: the value of the node
    - left_child: the left child of the node
    - right_child: the right child of the node
    """

    def __init__(self, value):
        """
        Initializes a new instance of the Node class.

        Args:
        - value: the value of the node
        """
        self.value = value
        self.left_child = None
        self.right_child = None


class BST:
    """
    A class representing a binary search tree.

    Attributes:
    - root: the root node of the tree
    """

    def __init__(self):
        """
        Initializes a new instance of the BST class.
        """
        self.root = None

    def add(self, value):
        """
        Adds a new node with the given value to the tree.

        Args:
        - value: the value of the node to add
        """
        if self.root is None:
            # The root does exist yet, create it
            self.root = Node(value)
        else:
            # Find the right place and insert new value
            self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        A helper method to recursively traverse the tree and find the correct position to add the new node.

        Args:
        - current_node: the current node to traverse
        - value: the value of the node to add
        """
        if value <= current_node.value:
            # Go to the left
            if current_node.left_child is None:
                current_node.left_child = Node(value)
            else:
                self._add_recursive(current_node.left_child, value)
        else:
            # Go to the right
            if current_node.right_child is None:
                current_node.right_child = Node(value)
            else:
                self._add_recursive(current_node.right_child, value)
    
    def _find_recursive(self, current_node, value):
        """
        A helper method to recursively traverse the tree and find the node with the given value.

        Args:
        - current_node: the current node to traverse
        - value: the value to search for

        Returns:
        - The node with the given value if found, None otherwise
        """
        if current_node is None:
            return None
        if current_node.value == value:
            return current_node
        if value < current_node.value:
            return self._find_recursive(current_node.left_child, value)
        return self._find_recursive(current_node.right_child, value)
    
    def find(self, value):
        """
        Finds the node with the given value in the tree.

        Args:
        - value: the value to search for

        Returns:
        - The node with the given value if found, None otherwise
        """
        return self._find_recursive(self.root, value)

    
    def _contains(self, current_node, value):
        """
        A helper method to recursively traverse the tree and find the node with the given value.

        Args:
        - current_node: the current node to traverse
        - value: the value to search for

        Returns:
        - True if a node with the given value is found, False otherwise
        """
        if current_node is None:
            return False
        if current_node.value == value:
            return True
        if value < current_node.value:
            return self._contains(current_node.left_child, value)
        return self._contains(current_node.right_child, value)

    def contains(self, value):
        """
        Checks whether a node with the given value is present in the tree.

        Args:
        - value: the value to search for

        Returns:
        - True if a node with the given value is found, False otherwise
        """
        return self._contains(self.root, value)

    def plot(self):
        """
        Plots the binary search tree using Plotly.
        """
        if self.root is None:
            print("The tree is empty!")
            return

        # Initialize lists for coordinates and connections
        node_coords = []
        lines = []

        # Helper function to traverse the tree and fill the coordinate and connection lists
        def _plot_recursive(node, x, y, offset):
            if node is not None:
                node_coords.append((x, y, node.value))
                if node.left_child is not None:
                    new_x = x - offset
                    new_y = y - 1
                    lines.append((x, y, new_x, new_y))
                    _plot_recursive(node.left_child, new_x, new_y, offset / 2)
                if node.right_child is not None:
                    new_x = x + offset
                    new_y = y - 1
                    lines.append((x, y, new_x, new_y))
                    _plot_recursive(node.right_child, new_x, new_y, offset / 2)

        # Traverse the tree starting from the root node
        _plot_recursive(self.root, x=0, y=0, offset=0.5)

        # Create a scatter plot for the nodes
        node_trace = go.Scatter(x=[x for x, y, _ in node_coords],
                                y=[y for _, y, _ in node_coords],
                                text=[str(val) for _, _, val in node_coords],
                                mode='markers+text',
                                textposition='top center',
                                marker=dict(symbol='circle',
                                            size=20,
                                            color='darkblue'))

        # Create a scatter plot for the connections between nodes
        line_trace = go.Scatter(x=sum([[x1, x2, None] for x1, y1, x2, y2 in lines], []),
                                y=sum([[y1, y2, None] for x1, y1, x2, y2 in lines], []),
                                mode='lines',
                                line=dict(color='black'))

        # Combine the two scatter plots
        layout = go.Layout(title='',
                           xaxis=dict(title='', showgrid=False, zeroline=False, showticklabels=False),
                           yaxis=dict(title='', showgrid=False, zeroline=False, showticklabels=False),
                           showlegend=False)

        fig = go.Figure(data=[node_trace, line_trace], layout=layout)
        fig.show()

# Example usage:

bst = BST()
for value in [5, 3, 1, 0, 2, 4, 7, 6, 8]:
    bst.add(value)
bst.plot()

Overwriting binarysearchtree.py


In [3]:
%%file validatethreenodes.py
import pytest
from binarysearchtree import *

def validateThreeNodes(bst, valueOne, valueTwo, valueThree):
    """
    Checks if the given three nodes have the required relationship in the Binary Search Tree.
    
    This function validates if either nodeTwo is a descendant of nodeOne and nodeThree is a descendant
    of nodeTwo, or if nodeOne is a descendant of nodeTwo and nodeThree is an ascendant of nodeTwo.

    Parameters:
    bst (BST): The Binary Search Tree containing the nodes.
    valueOne (int): The value of the first node.
    valueTwo (int): The value of the second node.
    valueThree (int): The value of the third node.

    Returns:
    bool: True if the given nodes have the required relationship, False otherwise.
    """
    # Find the nodes in the BST
    nodeOne = bst.find(valueOne)
    nodeTwo = bst.find(valueTwo)
    nodeThree = bst.find(valueThree)
    
    # Check if nodeOne is a descendant of nodeTwo and nodeThree is an ancestor of nodeTwo
    if is_descendant(nodeTwo, nodeOne) and is_ancestor(nodeTwo, nodeThree):
        return True
    
    # Check if nodeThree is a descendant of nodeTwo and nodeOne is an ancestor of nodeTwo
    if is_descendant(nodeTwo, nodeThree) and is_ancestor(nodeTwo, nodeOne):
        return True
    
    # If neither of the above conditions were met, the nodes don't have the required relationship
    return False


def is_ancestor(target, node):
    """
    Checks if target is an ancestor of node.
    
    Parameters:
    target (TreeNode): The node to search for.
    node (TreeNode): The node to start the search from.
    
    Returns:
    bool: True if target is an ancestor of node, False otherwise.
    """
    if node is None:
        return False
    
    if node.left_child == target or node.right_child == target:
        return True
    
    return is_ancestor(target, node.left_child) or is_ancestor(target, node.right_child)

def is_descendant(node, target):
    """
    Checks if target is a descendant of node.
    
    Parameters:
    node (TreeNode): The node to start the search from.
    target (TreeNode): The node to search for.
    
    Returns:
    bool: True if target is a descendant of node, False otherwise.
    """
    if node is None:
        return False
    
    if node == target:
        return True
    
    return is_descendant(node.left_child, target) or is_descendant(node.right_child, target)



@pytest.fixture(scope="session")
def data():
    
    array = [[5, 2, 1, 0, 4, 3, 7, 6, 8],
             [5, 3, 2, 1, 0, 4, 7, 6, 8],
             [5, 3, 2, 1, 0, 4, 7, 6, 8],
             [2, 1, 3, 4, 5, 6, 7, 8, 9],
             [6, 4, 3, 1, 2, 8, 7, 9],
             [2, 1, 3, 4],
             [8, 4, 3, 2, 1, 10, 9, 14, 12, 11, 6, 7, 13],
             [8, 7, 6, 5, 4, 3, 2, 1, 9],
             [3, 2, 1],
             [3, 2, 1],
             [6, 4, 2, 1, 3, 5, 8, 7, 9],
             [10, 6, 5, 3, 1, 2, 4, 8, 7, 9, 15, 14, 13, 11, 12, 18, 17, 16],
             [10, 6, 5, 3, 1, 2, 4, 8, 7, 9, 15, 14, 13, 11, 12, 18, 17, 16],
             [5, 3, 1, 0, 2, 4, 7, 6, 8],
             [5, 3, 1, 0, 2, 4, 7, 6, 8]]
    return array

def test_1(data):
    """
    Test evaluation for "nodeOne": "5","nodeThree": "3","nodeTwo": "2"
    """
    bst = BST()
    for value in data[0]:
      bst.add(value)
    assert validateThreeNodes(bst,5,2,3) == True

def test_2(data):
    """
    Test evaluation for "nodeOne": "5", "nodeThree": "1", "nodeTwo": "8",
    """
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert validateThreeNodes(bst,5,8,1) == False

def test_3(data):
    """
    Test evaluation for   "nodeOne": "8","nodeThree": "2","nodeTwo": "5",
    """
    bst = BST()
    for value in data[2]:
      bst.add(value)
    assert validateThreeNodes(bst,8,5,2) == False

def test_4(data):
    """
    Test evaluation for  "nodeOne": "2","nodeThree": "8","nodeTwo": "5"
    """
    bst = BST()
    for value in data[3]:
      bst.add(value)
    assert validateThreeNodes(bst,2,5,8) == True

def test_5(data):
    """
    Test evaluation for "nodeOne": "4", "nodeThree": "2", "nodeTwo": "1",
    """
    bst = BST()
    for value in data[4]:
      bst.add(value)
    assert validateThreeNodes(bst,4,1,2) == True

def test_6(data):
    """
    Test evaluation for "nodeOne": "1","nodeThree": "3","nodeTwo": "2",
    """
    bst = BST()
    for value in data[5]:
      bst.add(value)
    assert validateThreeNodes(bst,1,2,3) == False

def test_7(data):
    """
    Test evaluation for "nodeOne": "2","nodeThree": "13","nodeTwo": "4"
    """
    bst = BST()
    for value in data[6]:
      bst.add(value)
    assert validateThreeNodes(bst,2,4,13) == False

def test_8(data):
    """
    Test evaluation for "nodeOne": "8","nodeThree": "1","nodeTwo": "7"
    """
    bst = BST()
    for value in data[7]:
      bst.add(value)
    assert validateThreeNodes(bst,8,7,1) == True

def test_9(data):
    """
    Test evaluation for "nodeOne": "2","nodeThree": "3","nodeTwo": "1"
    """
    bst = BST()
    for value in data[8]:
      bst.add(value)
    assert validateThreeNodes(bst,2,1,3) == False

def test_10(data):
    """
    Test evaluation for "nodeOne": "1", "nodeThree": "3", "nodeTwo": "2"
    """
    bst = BST()
    for value in data[9]:
      bst.add(value)
    assert validateThreeNodes(bst,1,2,3) == True

def test_11(data):
    """
    Test evaluation for "nodeOne": "9","nodeThree": "6","nodeTwo": "8"
    """
    bst = BST()
    for value in data[10]:
      bst.add(value)
    assert validateThreeNodes(bst,9,8,6) == True

def test_12(data):
    """
    Test evaluation for "nodeOne": "12","nodeThree": "15","nodeTwo": "13"
    """
    bst = BST()
    for value in data[11]:
      bst.add(value)
    assert validateThreeNodes(bst,12,13,15) == True

def test_13(data):
    """
    Test evaluation for "nodeOne": "5","nodeThree": "15","nodeTwo": "10"
    """
    bst = BST()
    for value in data[12]:
      bst.add(value)
    assert validateThreeNodes(bst,5,10,15) == False

def test_14(data):
    """
    Test evaluation for "nodeOne": "5","nodeThree": "4","nodeTwo": "3"
    """
    bst = BST()
    for value in data[13]:
      bst.add(value)
    assert validateThreeNodes(bst,5,3,4) == True

def test_15(data):
    """
    Test evaluation for "nodeOne": "5","nodeThree": "1","nodeTwo": "3"
    """
    bst = BST()
    for value in data[14]:
      bst.add(value)
    assert validateThreeNodes(bst,5,3,1) == True

Overwriting validatethreenodes.py


In [4]:
!pytest validatethreenodes.py -vv

============================= test session starts =============================
platform win32 -- Python 3.10.9, pytest-7.1.2, pluggy-1.0.0 -- C:\Users\gabri\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\gabri
plugins: anyio-3.5.0, sugar-0.9.6
collecting ... collected 15 items

validatethreenodes.py::test_1 PASSED                                     [  6%]
validatethreenodes.py::test_2 PASSED                                     [ 13%]
validatethreenodes.py::test_3 PASSED                                     [ 20%]
validatethreenodes.py::test_4 PASSED                                     [ 26%]
validatethreenodes.py::test_5 PASSED                                     [ 33%]
validatethreenodes.py::test_6 PASSED                                     [ 40%]
validatethreenodes.py::test_7 PASSED                                     [ 46%]
validatethreenodes.py::test_8 PASSED                                     [ 53%]
validatethreenodes.py::test_9 PASSED                            

In [24]:
#%%file validatethreenodes.py
import pytest
from binarysearchtree import *

def validateThreeNodes(bst, valueOne, valueTwo, valueThree):#IGNORE DEBUGGING 
    """
    Checks if the given three nodes have the required relationship in the Binary Search Tree.
    
    This function validates if either nodeTwo is a descendant of nodeOne and nodeThree is a descendant
    of nodeTwo, or if nodeOne is a descendant of nodeTwo and nodeThree is an ascendant of nodeTwo.

    Parameters:
    bst (BST): The Binary Search Tree containing the nodes.
    valueOne (int): The value of the first node.
    valueTwo (int): The value of the second node.
    valueThree (int): The value of the third node.

    Returns:
    bool: True if the given nodes have the required relationship, False otherwise.
    """
    # Find the nodes in the BST
    nodeOne = bst.find(valueOne)
    nodeTwo = bst.find(valueTwo)
    nodeThree = bst.find(valueThree)
    print('nó 1:',nodeOne.value,'nó 2:',nodeTwo.value,'nó 3:',nodeThree.value)
    # Check if nodeOne is a descendant of nodeTwo and nodeThree is an ancestor of nodeTwo
    if is_descendant(nodeTwo, nodeOne) and is_ancestor(nodeTwo, nodeThree):
        return True
    
    # Check if nodeThree is a descendant of nodeTwo and nodeOne is an ancestor of nodeTwo
    if is_descendant(nodeTwo, nodeThree) and is_ancestor(nodeTwo, nodeOne):
        return True
    
    # If neither of the above conditions were met, the nodes don't have the required relationship
    return False


def is_ancestor(node, target):
    """
    Checks if target is an ancestor of node.
    
    Parameters:
    node (TreeNode): The node to start the search from.
    target (TreeNode): The node to search for.
    
    Returns:
    bool: True if target is an ancestor of node, False otherwise.
    """
    if node is None:
        return False
    
    if node == target:
        
        print(node.value,'is ancestor of ',target.value)
       
        return True
    
    return is_ancestor(node.left_child, target) or is_ancestor(node.right_child, target)

def is_descendant(node, target):
    """
    Checks if target is a descendant of node.
    
    Parameters:
    node (TreeNode): The node to start the search from.
    target (TreeNode): The node to search for.
    
    Returns:
    bool: True if target is a descendant of node, False otherwise.
    """
    if node is None:
        return False
    
    if node == target:
        print(node.value,'is descendant of ',target.value)
        return True
    
    return is_descendant(node.left_child, target) or is_descendant(node.right_child, target)




def data():
    
    array = [[5, 2, 1, 0, 4, 3, 7, 6, 8],
             [5, 3, 2, 1, 0, 4, 7, 6, 8],
             [5, 3, 2, 1, 0, 4, 7, 6, 8],
             [2, 1, 3, 4, 5, 6, 7, 8, 9],
             [6, 4, 3, 1, 2, 8, 7, 9],
             [2, 1, 3, 4],
             [8, 4, 3, 2, 1, 10, 9, 14, 12, 11, 6, 7, 13],
             [8, 7, 6, 5, 4, 3, 2, 1, 9],
             [3, 2, 1],
             [3, 2, 1],
             [6, 4, 2, 1, 3, 5, 8, 7, 9],
             [10, 6, 5, 3, 1, 2, 4, 8, 7, 9, 15, 14, 13, 11, 12, 18, 17, 16],
             [10, 6, 5, 3, 1, 2, 4, 8, 7, 9, 15, 14, 13, 11, 12, 18, 17, 16],
             [5, 3, 1, 0, 2, 4, 7, 6, 8],
             [5, 3, 1, 0, 2, 4, 7, 6, 8]]
    return array

def test_1(data):
    """
    Test evaluation for "nodeOne": "5","nodeThree": "3","nodeTwo": "2"
    """
    bst = BST()
    for value in data[0]:
      bst.add(value)
    assert validateThreeNodes(bst,5,2,3) == True

def test_2(data):
    """
    Test evaluation for "nodeOne": "5", "nodeThree": "1", "nodeTwo": "8",
    """
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert validateThreeNodes(bst,5,8,1) == False
array = [5, 2, 1, 0, 4, 3, 7, 6, 8]
bst = BST()
for value in array:
    bst.add(value)
    print(value)
print(validateThreeNodes(bst,5,2,3))

5
2
1
0
4
3
7
6
8
nó 1: 5 nó 2: 2 nó 3: 3
3 is descendant of  3
False
